In [1]:
"""
This cell writes the desired class labels into files
"""
# import numpy as np
# import os

# class_idx_dir = "/home/zl310/cs585_project/vmoe/chosen_class_idx/"
# rng = np.random.default_rng()
# num_total_classes = 1000
# # num_chosen_classes_list = [2, 4, 8, 16, 32, 64]
# num_chosen_classes_list = [128, 256, 512, 1000]

# for num_chosen_classes in num_chosen_classes_list:
#     class_idx = np.array(range(num_total_classes))
#     chosen_class_idx = rng.permutation(class_idx)[0:num_chosen_classes]
#     np.save(os.path.join(class_idx_dir, f"n_{num_chosen_classes}.npy"), chosen_class_idx)

# import numpy as np
# import os

# class_idx_dir = "/home/zl310/cs585_project/vmoe/chosen_class_idx/"
# num_chosen_classes_list = [2, 4, 8, 16, 32, 64]

# for num_chosen_classes in num_chosen_classes_list:
#     test_file = np.load(os.path.join(class_idx_dir, f"n_{num_chosen_classes}.npy"))
#     print(test_file)

In [4]:
import numpy as np
import os

"""
example names: 

x_0ac27bf3-88a3-4a7f-88b2-22ec54f8d68e_batch_174_layer_5.npy
x_0ac27bf3-88a3-4a7f-88b2-22ec54f8d68e_batch_174_layer_7.npy
y_0ac27bf3-88a3-4a7f-88b2-22ec54f8d68e_batch_174_layer_both.npy

shapes: 
x: (256, 12, 12, 2)
y: (256,)

num_chosen_classes_list should be [2, 4, 8, 16, 32, 64]

there are 732-1300 images per class in the ILSVRC2012 training set

This cell filters the dataset given the desired classes
"""


def dataset_from_classes(orig_data_dir, uuid_batch_indices, which_classes: set):
    x5_list, x7_list, y_list = [], [], []
    for id in uuid_batch_indices:
        x5 = np.load(os.path.join(orig_data_dir, f"x_{id}_layer_5.npy"))
        x7 = np.load(os.path.join(orig_data_dir, f"x_{id}_layer_7.npy"))
        y = np.load(os.path.join(orig_data_dir, f"y_{id}_layer_both.npy"))
        if len(y) != x5.shape[0]:
            y = y[0:x5.shape[0]]
            print(f"len(y) != x.shape[0], len(y) = {len(y)}, x5.shape[0] = {x5.shape[0]}")
        for batch_i in range(len(y)):
            if y[batch_i] in which_classes:
                x5_list.append(x5[batch_i])
                x7_list.append(x7[batch_i])
                y_list.append(y[batch_i])
    return x5_list, x7_list, y_list

def write_new_data(train_or_test, which_n_classes):
    class_idx_dir = "/home/zl310/cs585_project/vmoe/chosen_class_idx/"
    orig_data_dir = f"/home/zl310/cs585_project/vmoe/expert_assign_{train_or_test}_ImageNetData/"
    dest_dir = f"/home/zl310/cs585_project/vmoe/{train_or_test}_data_selected_classes/"
    class_idx_files = os.listdir(class_idx_dir)

    files = os.listdir(orig_data_dir)
    uuid_b_idx = set()
    for file in files:
        split = file.split("_")
        uuid_b_idx.add('_'.join(split[1:4]))

    for f_name in class_idx_files:
        which_classes = np.load(os.path.join(class_idx_dir, f_name))
        if len(which_classes) in which_n_classes:
            x5_list, x7_list, y_list = dataset_from_classes(orig_data_dir, uuid_b_idx, set(which_classes))
            this_dest_dir = os.path.join(dest_dir, f"data_n_{len(which_classes)}")
            os.makedirs(this_dest_dir, exist_ok=True)
            x5_arr, x7_arr, y_arr = np.stack(x5_list), np.stack(x7_list), np.array(y_list)
            np.save(os.path.join(this_dest_dir, f"x5_n_{len(which_classes)}.npy"), x5_arr)
            np.save(os.path.join(this_dest_dir, f"x7_n_{len(which_classes)}.npy"), x7_arr)
            np.save(os.path.join(this_dest_dir, f"y_n_{len(which_classes)}.npy"), y_arr)

def validate_new_data(train_or_test):
    class_idx_dir = "/home/zl310/cs585_project/vmoe/chosen_class_idx/"
    dest_dir = f"/home/zl310/cs585_project/vmoe/{train_or_test}_data_selected_classes/"
    class_idx_files = os.listdir(class_idx_dir)
    for f_name in class_idx_files:
        which_classes = np.load(os.path.join(class_idx_dir, f_name))
        this_dest_dir = os.path.join(dest_dir, f"data_n_{len(which_classes)}")
        print(f"x5, {len(which_classes)}")
        print(np.load(os.path.join(this_dest_dir, f"x5_n_{len(which_classes)}.npy")).shape)
        print("x7:")
        print(np.load(os.path.join(this_dest_dir, f"x7_n_{len(which_classes)}.npy")).shape)
        print("y:")
        print(np.load(os.path.join(this_dest_dir, f"y_n_{len(which_classes)}.npy")).shape)


write_new_data(train_or_test='test', which_n_classes=set([128, 256, 512, 1000]))
validate_new_data(train_or_test='train')

len(y) != x.shape[0], len(y) = 80, x5.shape[0] = 80
len(y) != x.shape[0], len(y) = 80, x5.shape[0] = 80
len(y) != x.shape[0], len(y) = 80, x5.shape[0] = 80
len(y) != x.shape[0], len(y) = 80, x5.shape[0] = 80
x5, 4
(5150, 12, 12, 2)
x7:
(5150, 12, 12, 2)
y:
(5150,)
x5, 64
(82155, 12, 12, 2)
x7:
(82155, 12, 12, 2)
y:
(82155,)
x5, 2
(2432, 12, 12, 2)
x7:
(2432, 12, 12, 2)
y:
(2432,)
x5, 128
(163372, 12, 12, 2)
x7:
(163372, 12, 12, 2)
y:
(163372,)
x5, 512
(649170, 12, 12, 2)
x7:
(649170, 12, 12, 2)
y:
(649170,)
x5, 16
(20583, 12, 12, 2)
x7:
(20583, 12, 12, 2)
y:
(20583,)
x5, 8
(10289, 12, 12, 2)
x7:
(10289, 12, 12, 2)
y:
(10289,)
x5, 32
(40195, 12, 12, 2)
x7:
(40195, 12, 12, 2)
y:
(40195,)
x5, 256
(325616, 12, 12, 2)
x7:
(325616, 12, 12, 2)
y:
(325616,)
x5, 1000
(1268355, 12, 12, 2)
x7:
(1268355, 12, 12, 2)
y:
(1268355,)


In [2]:
import numpy as np
import os


def compute_statistics(which_n_classes, dir_path="/home/zl310/cs585_project/vmoe/train_data_selected_classes/"):
    files = os.listdir(dir_path)
    class_idx_dir = "/home/zl310/cs585_project/vmoe/chosen_class_idx/"
    class_idx_files = os.listdir(class_idx_dir)
    for f_name in class_idx_files:
        which_classes = np.load(os.path.join(class_idx_dir, f_name))
        if len(which_classes) in which_n_classes:
            this_dest_dir = os.path.join(dir_path, f"data_n_{len(which_classes)}")
            x5 = np.load(os.path.join(this_dest_dir, f"x5_n_{len(which_classes)}.npy"))
            x7 = np.load(os.path.join(this_dest_dir, f"x7_n_{len(which_classes)}.npy"))
            print(f"std num classes = {len(set(which_classes))}", np.std(np.concatenate((x5.flatten(), x7.flatten()))))
            print(np.mean(np.concatenate((x5.flatten(), x7.flatten()))))
    return

compute_statistics(which_n_classes=set([128, 256, 512, 1000]))  # this is used in model_unencrypted/dataset_utils

std num classes = 128 2.3167054615253844
4.517264099638317
std num classes = 512 2.299420980057555
4.499429311338581
std num classes = 256 2.2901221928285342
4.485078938202259
std num classes = 1000 2.303071876769264
4.495041399588444
